### Getting started with leanclient

leanclient allows for easy interaction with the [lean4 theorem prover](https://github.com/leanprover/lean4) using the [Language Server Protocol](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/).

This notebook shows a minimal example for how to setup and use leanclient.

- leanclient does **not create/manage lean projects**. You should create your own project, e.g. in vscode, and then point leanclient at the root folder (where lakefile.toml is located). See below for an example setup.

- Running this on Google Colab is **slow for larger projects** (e.g. including mathlib). In general, performance is reduced in notebooks.

Check out the [documentation](https://leanclient.readthedocs.io/en/latest/) and the [github repository](https://github.com/oOo0oOo/leanclient) for more information.

In [1]:
PROJECT_NAME = "LeanProject"
PROJECT_PATH = PROJECT_NAME + "/"
LEAN_VERSION = "v4.14.0"

In [2]:
# Setup elan, lean, and a new project
!curl https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh -sSf | sh -s -- -y

import os
os.environ["PATH"] = os.path.expanduser("~/.elan/bin") + ":" + os.environ["PATH"]

!elan toolchain install leanprover/lean4:{LEAN_VERSION}
!elan default leanprover/lean4:{LEAN_VERSION}

!lake new {PROJECT_NAME}
%cd {PROJECT_NAME}
!lake build
%cd ..

info: downloading installer
info: syncing channel updates for 'stable'
info: latest update on stable, lean version v4.15.0
info: downloading component 'lean'
252.6 MiB / 252.6 MiB (100 %) 105.2 MiB/s ETA:   0 s
info: installing component 'lean'
info: default toolchain set to 'stable'

  stable installed - Lean (version 4.15.0, x86_64-unknown-linux-gnu, commit 11651562caae, Release)

info: downloading component 'lean'
238.3 MiB / 238.3 MiB (100 %) 178.7 MiB/s ETA:   0 s
info: installing component 'lean'

  leanprover/lean4:v4.14.0 installed - Lean (version 4.14.0, x86_64-unknown-linux-gnu, commit 410fab728470, Release)

info: using existing install for 'leanprover/lean4:v4.14.0'
info: default toolchain set to 'leanprover/lean4:v4.14.0'

  leanprover/lean4:v4.14.0 unchanged - Lean (version 4.14.0, x86_64-unknown-linux-gnu, commit 410fab728470, Release)

/content/LeanProject
info: LeanProject: no previous manifest, creating one from scratch
info: toolchain not updated; already up-to-date


In [3]:
# Install leanclient
!pip install leanclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: orjson
    Found existing installation: orjson 3.10.12
    Uninstalling orjson-3.10.12:
      Successfully uninstalled orjson-3.10.12


In [4]:
from pprint import pprint
import leanclient as lc

# Create client and point it to the root folder of the project.
# This is where lakefile.toml is located.
client = lc.LeanLSPClient(PROJECT_PATH)

# Use a SingleFileClient for simple interactions with a file.
# This file is a very simple template created during project init.
sfc = client.create_file_client("LeanProject/Basic.lean")

# Query the language server!
symbols = sfc.get_document_symbol()
tokens = sfc.get_semantic_tokens()
term_goal = sfc.get_term_goal(0, 20)

print("Symbols:")
pprint(symbols)

print("\nSemantic tokens:")
pprint(tokens)

print("\nTerm goal:")
pprint(term_goal)

# Always close the client when you're done.
client.close()

Symbols:
[{'kind': 6,
  'name': 'hello',
  'range': {'end': {'character': 20, 'line': 0},
            'start': {'character': 0, 'line': 0}},
  'selectionRange': {'end': {'character': 9, 'line': 0},
                     'start': {'character': 4, 'line': 0}}}]

Semantic tokens:
[[0, 0, 3, 'keyword']]

Term goal:
{'goal': '⊢ String',
 'range': {'end': {'character': 20, 'line': 0},
           'start': {'character': 13, 'line': 0}}}


In [5]:
# Batch processing of files in parallel (multiprocessing) is easy using the LeanClientPool.

files = ["LeanProject/Basic.lean", "Main.lean"]

# Define a function which accepts a SingleFileClient as its only parameter.
def count_tokens(client: lc.SingleFileClient):
  return len(client.get_semantic_tokens())

with lc.LeanClientPool(PROJECT_PATH, num_workers=2) as pool:
  results = pool.map(count_tokens, files)

  # You can also use pool.submit() for increased control
  futures = [pool.submit(count_tokens, path) for path in files]
  fut_res = [f.get() for f in futures]

print("Number of tokens in each file:")
pprint(results)

Number of tokens in each file:
[1, 3]


### All done!

Change the contents of the file (click the folder on the left) `LeanProject/LeanProject/Basic.lean` and rerun the code above.

For example, you could add this theorem:

```lean
-- Define a simple theorem: for any natural number n, n + 0 = n
theorem add_zero_custom (n : Nat) : n + 0 = n := by
  -- Use induction on n
  induction n with
  | zero =>
    -- Base case: 0 + 0 = 0
    rfl
  | succ n' ih =>
    -- Inductive step: assume n' + 0 = n', prove (n' + 1) + 0 = (n' + 1)
    rw [Nat.add_succ, ih]
```

### Next

Check out the [documentation](https://leanclient.readthedocs.io/en/latest/) for more information on leanclient!